In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler #scikit-learn==0.21.3
from xgboost import XGBClassifier #xgboost==0.90
from sklearn.manifold import TSNE
import xgboost
from sklearn.preprocessing import LabelEncoder

In [2]:
xgboost.__version__

'1.7.4'

## G1 predictions

In [3]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

G1_TPM_merged_lats = pd.read_csv('../../g1Surface/G1_surface_allSamples_processed_updatedMarferret_marmicroDb2023_noOutliers_tpm_noNAPfam_fall2023_noNATaxa.csv')

In [4]:
#some pfams still have 0 counts?
G1_TPM_merged_lats.columns[(G1_TPM_merged_lats == 0).all()]

Index([], dtype='object')

In [5]:
features = features['pfam']

In [6]:
len(features)

183

In [7]:
original_features = set(G1_TPM_merged_lats.columns)

In [8]:
missing_features = set(features) - original_features
missing_features

{'PF14259'}

In [9]:
#Add the missing features to G1_TPM_merged_lats, filled with 0 values
for feature in missing_features:
    G1_TPM_merged_lats[feature] = 0

In [10]:
# Initialize empty DataFrames to store results
excluded_pfams = pd.DataFrame()

output_predictions = pd.DataFrame()

In [11]:
for i in range (0, 30):
    # Assuming G1_TPM_merged_lats is your DataFrame
    df = G1_TPM_merged_lats.copy()

    # Calculate 10% of the columns
    num_cols = df.shape[1]
    num_cols_to_zero = int(num_cols * 0.1)

    # Randomly select 10% of the columns
    columns_to_zero = np.random.choice(df.columns, size=num_cols_to_zero, replace=False)

    # Replace the selected columns with all zero values
    df[columns_to_zero] = 0

    # df now has 10% of the columns replaced with zeroes

    excluded_pfams[f'col{i}'] = columns_to_zero

    df = df[features]

    le = LabelEncoder()

    train_data = train_data[features]

    X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    #{'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'reg_lambda': 1.0}
    model = XGBClassifier(gamma=0.0, learning_rate=0.1, max_depth=3, n_estimators=1000, reg_lambda=1.0)
    model.fit(X,y)

    df = scaler.transform(df)
    xg_predictions = model.predict(df)
    output_predictions[f'col{i}'] = xg_predictions

In [12]:
output_predictions.to_csv('/Users/elainathomas/Dropbox/grad/research/g1Surface/bootstrapPredictions.csv')

In [13]:
excluded_pfams.to_csv('/Users/elainathomas/Dropbox/grad/research/g1Surface/bootstrapPfamsExcluded.csv')